# Final Project


* Read in Seattle Crime Dataset

In [1]:
import pandas as pd
file_name = 'SPD_Reports.csv'
crime_df = pd.read_csv(file_name)

* Drop Irrelevant Columns ('Offense Type', 'Offense End Date', 'Block', 'District', '2000 Census Tract', 'Longitude',     'Latitude', 'Report Date')

* Reorder Columns to read 'Offense Start Date' -> 'Offense Description' -> 'Beat'

* Rename 'Offense Start Date' to 'Offense Date'; just for readability


In [2]:
crime_df=crime_df.drop(columns=['Offense Type', 'Offense End Date', 'Block', 'District', '2000 Census Tract', 'Longitude', 'Latitude', 'Report Date'])
crime_df = crime_df[['Offense Start Date', 'Offense Description', 'Beat']]
crime_df.rename(columns = {'Offense Start Date':'Offense Date'}, inplace = True) 

* Convert 'Offense Date' to datetime
* Sort crime_df by ascending date (2007 - 2016(

In [3]:
crime_df['Offense Date'] = pd.to_datetime(crime_df['Offense Date'])
crime_df = crime_df.sort_values(by='Offense Date')

* Read in dataset from "Did in rain in Seattle?" 

* Convert Date feature to datetime object

* Convert Rain feature to bool object

In [4]:
f_name = 'seattleWeather_1948-2017.csv'
weather_df = pd.read_csv(f_name)
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
weather_df['RAIN'] = weather_df['RAIN'].astype('bool')

* Filter both the crime_df and weather_df by year (2007 - 2016)

In [5]:
weather_df = weather_df[(weather_df['DATE'].dt.year >= 2007) & 
                        ((weather_df['DATE'].dt.year <= 2016) & (weather_df['DATE'].dt.month <= 9) & (weather_df['DATE'].dt.day <= 19))]
crime_df = crime_df[(crime_df['Offense Date'].dt.year >= 2007) & 
                    ((crime_df['Offense Date'].dt.year <= 2017))]


* crime_df['Offense Date'] had a time value attached to it; first line of code just drops the time but keeps the date (So that it's easier to work with the weather_df)

* Index of crime_df is changed to the date of incident

* Index of weather_df is also changed to the date feature 

* Join the two dataframes together according to index (the date) 

    So if the dates match, the values from weather_df are assigned correctly 

In [6]:
crime_df['Offense Date'] = crime_df['Offense Date'].dt.date
crime_df = crime_df.set_index('Offense Date')
weather_df = weather_df.set_index('DATE')
new_crime_df = crime_df.join(weather_df, lsuffix='_caller', rsuffix='_other')

In [7]:
new_crime_df.head()

,Offense Description,Beat,PRCP,TMAX,TMIN,RAIN
2007-01-01,FRAUD,F2,0.41,48.0,34.0,True
2007-01-01,VEHICLE THEFT,J1,0.41,48.0,34.0,True
2007-01-01,FRAUD,F2,0.41,48.0,34.0,True
2007-01-01,OTHER PROPERTY,G2,0.41,48.0,34.0,True
2007-01-01,FRAUD,E1,0.41,48.0,34.0,True


Below I am getting the total number of Offenses on any given day and adding it as a column to the new_crime_df

.

.

In [8]:
df = new_crime_df

This gets the count of each index, since each row is a crime if there's multiple indexes with the same date those get summed together for the total number of offenses for the day

In [9]:
index_count = pd.Index(df.index).value_counts()
df = index_count.to_frame().sort_index()

Above produces a series which we then convert to a dataframe and sort the index into ascending order by date

Now that the new_crime_df and this df have a similar index we can join the columns from both together

In [10]:
new_crime_df = new_crime_df.join(df, lsuffix='_caller', rsuffix='_other')

Below just changes the name of the columns for clarity

In [11]:
new_crime_df.columns = ['Offense Description', 'Beat', 'PRCP', 'TMAX', 'TMIN', 'RAIN', '# of Offenses in day']

In [12]:
new_crime_df

,Offense Description,Beat,PRCP,TMAX,TMIN,RAIN,# of Offenses in day
2007-01-01,FRAUD,F2,0.41,48.0,34.0,True,46
2007-01-01,VEHICLE THEFT,J1,0.41,48.0,34.0,True,46
2007-01-01,FRAUD,F2,0.41,48.0,34.0,True,46
2007-01-01,OTHER PROPERTY,G2,0.41,48.0,34.0,True,46
2007-01-01,FRAUD,E1,0.41,48.0,34.0,True,46
...,...,...,...,...,...,...,...
2016-09-19,ASSAULT,C3,0.08,68.0,53.0,True,13
2016-09-19,OTHER PROPERTY,U3,0.08,68.0,53.0,True,13
2016-09-19,SHOPLIFTING,L3,0.08,68.0,53.0,True,13
2016-09-19,ASSAULT,U2,0.08,68.0,53.0,True,13


Below I'm creating another dataframe execept this df is downsampled to just be the rain info and number of offenses per day

In [13]:
perDay_df = new_crime_df.resample('D').mean()
perDay_df

,PRCP,TMAX,TMIN,# of Offenses in day
2007-01-01,0.41,48.0,34.0,46.0
2007-01-02,NaN,NaN,NaN,NaN
2007-01-03,NaN,NaN,NaN,NaN
2007-01-04,0.15,42.0,30.0,3.0
2007-01-05,0.77,43.0,31.0,3.0
...,...,...,...,...
2016-09-15,0.00,72.0,52.0,264.0
2016-09-16,0.00,74.0,52.0,213.0
2016-09-17,0.22,67.0,57.0,192.0
2016-09-18,0.00,68.0,54.0,117.0
